# Import thư viện cần thiết

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing  import StandardScaler
from torch.utils.data import TensorDataset, DataLoader


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tracemalloc
import json


# Đọc dữ liệu đầu vào

In [3]:
data_path='../../dataset'

In [4]:
with open(data_path + '/train.json', 'r') as f:
    train_data = json.load(f)
with open(data_path + '/test.json', 'r') as f:
    test_data = json.load(f)

# Tiền xử lý dữ liệu

In [5]:
train_data= pd.DataFrame(train_data)
test_data= pd.DataFrame(test_data)

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195 entries, 0 to 1194
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   audio_embedding                  1195 non-null   object
 1   is_turkey                        1195 non-null   int64 
 2   vid_id                           1195 non-null   object
 3   end_time_seconds_youtube_clip    1195 non-null   int64 
 4   start_time_seconds_youtube_clip  1195 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 46.8+ KB


In [7]:
train_embeddings=train_data['audio_embedding'].apply(np.array)
max_len = train_embeddings.apply(len).max()
embedding_dim = len(train_embeddings.iloc[0][0])
train_X = pad_sequences(train_embeddings, maxlen=max_len, dtype='float32', padding='post', truncating='post')
train_Y = train_data['is_turkey'].values

In [8]:
valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_embeddings = test_data.loc[valid_idx, 'audio_embedding'].apply(np.array)
test_X = pad_sequences(test_embeddings, maxlen=max_len, dtype='float32', padding='post', truncating='post')

In [9]:
B, T, D = train_X.shape
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X.reshape(-1, D)).reshape(B, T, D)
test_X = scaler.transform(test_X.reshape(-1, D)).reshape(test_X.shape[0], T, D)

# Huấn luyện mô hình

In [10]:
# Bước 1: Split gốc
train_X_, val_X, train_Y_, val_Y = train_test_split(train_X, train_Y, test_size=0.3, random_state=45)

# Bước 2: Augment **chỉ tập train**
noise = np.random.normal(0, 0.01, size=train_X_.shape)
train_X_noisy = train_X_ + noise

# Bước 3: Nối lại tập train mở rộng
X_tr_aug = np.concatenate([train_X_, train_X_noisy], axis=0)
y_tr_aug = np.concatenate([train_Y_, train_Y_], axis=0)

# Bước 4: reshape & convert sang Tensor
X_tr_cnn = torch.tensor(X_tr_aug.reshape(-1, T, D), dtype=torch.float32)
y_tr = torch.tensor(y_tr_aug, dtype=torch.float32)

X_val_cnn = torch.tensor(val_X.reshape(-1, T, D), dtype=torch.float32)
y_val = torch.tensor(val_Y, dtype=torch.float32)


In [11]:
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        w = self.pool(x).squeeze(-1)
        w = self.fc(w).unsqueeze(-1)
        return x * w

class AudioCNNWithSE(nn.Module):
    def __init__(self, in_channels=128, out_channels=64, fc_out=1):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, 3, padding=1)
        self.se1 = SEBlock(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels * 2, 3, padding=1)
        self.se2 = SEBlock(out_channels * 2)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(out_channels * 2, fc_out)
        self.drop = nn.Dropout(0.3)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.drop(self.se1(F.relu(self.conv1(x))))
        x = self.drop(self.se2(F.relu(self.conv2(x))))
        x = self.pool(x).squeeze(-1)
        x = self.fc(x)
        return x


In [12]:
model = AudioCNNWithSE()  # or num_classes=3 if 3 classes
criterion = nn.BCEWithLogitsLoss()  # dùng nếu output là sigmoid (nhị phân)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5,weight_decay=1e-3)


In [13]:
best_val_loss = float('inf')
patience = 5
counter = 0

train_dataset = TensorDataset(X_tr_cnn, y_tr)
val_dataset = TensorDataset(X_val_cnn, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

for epoch in range(300):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(xb).squeeze()
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)
    avg_train_loss = running_loss / len(train_loader.dataset)
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            val_outputs = model(xb).squeeze()
            loss = criterion(val_outputs, yb)
            val_loss += loss.item() * xb.size(0)
        avg_val_loss = val_loss / len(val_loader.dataset)
        print(f"Epoch {epoch}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break


Epoch 0, Train Loss: 0.6926, Val Loss: 0.6915
Epoch 1, Train Loss: 0.6908, Val Loss: 0.6897
Epoch 2, Train Loss: 0.6892, Val Loss: 0.6878
Epoch 3, Train Loss: 0.6872, Val Loss: 0.6858
Epoch 4, Train Loss: 0.6853, Val Loss: 0.6836
Epoch 5, Train Loss: 0.6831, Val Loss: 0.6812
Epoch 6, Train Loss: 0.6809, Val Loss: 0.6786
Epoch 7, Train Loss: 0.6780, Val Loss: 0.6755
Epoch 8, Train Loss: 0.6748, Val Loss: 0.6720
Epoch 9, Train Loss: 0.6714, Val Loss: 0.6682
Epoch 10, Train Loss: 0.6672, Val Loss: 0.6638
Epoch 11, Train Loss: 0.6630, Val Loss: 0.6590
Epoch 12, Train Loss: 0.6580, Val Loss: 0.6536
Epoch 13, Train Loss: 0.6525, Val Loss: 0.6474
Epoch 14, Train Loss: 0.6462, Val Loss: 0.6407
Epoch 15, Train Loss: 0.6392, Val Loss: 0.6332
Epoch 16, Train Loss: 0.6315, Val Loss: 0.6251
Epoch 17, Train Loss: 0.6237, Val Loss: 0.6163
Epoch 18, Train Loss: 0.6150, Val Loss: 0.6069
Epoch 19, Train Loss: 0.6054, Val Loss: 0.5967
Epoch 20, Train Loss: 0.5952, Val Loss: 0.5861
Epoch 21, Train Loss: 0

In [14]:
from sklearn.metrics import roc_auc_score

# Đánh giá mô hình trên tập validation
model.eval()
with torch.no_grad():
    val_outputs = model(X_val_cnn).squeeze()
    val_probs = torch.sigmoid(val_outputs).cpu().numpy()
    val_preds = (val_probs > 0.5)
    y_true = y_val.cpu().numpy()
    acc = accuracy_score(y_true, val_preds)
    f1 = f1_score(y_true, val_preds)
    precision = precision_score(y_true, val_preds)
    recall = recall_score(y_true, val_preds)
    auc = roc_auc_score(y_true, val_probs)
    print(f"Validation AUC: {auc:.4f}")
    print(f"Validation Precision: {precision:.4f}")
    print(f"Validation Accuracy: {acc:.4f}")
    print(f"Validation F1 Score: {f1:.4f}")
    print(f"Validation Recall: {recall:.4f}")


Validation AUC: 0.9933
Validation Precision: 0.9441
Validation Accuracy: 0.9554
Validation F1 Score: 0.9441
Validation Recall: 0.9441


In [15]:
model.eval()
with torch.no_grad():
    test_tensor = torch.tensor(test_X, dtype=torch.float32)
    test_outputs = model(test_tensor).squeeze()
    test_probs = torch.sigmoid(test_outputs).cpu().numpy()

# Save probabilities to result.csv
test_data.loc[valid_idx, 'is_turkey'] = test_probs
test_data.loc[valid_idx, ['vid_id', 'is_turkey']].to_csv('result.csv', index=False)
